# Goal Programming Example

|          |       A       |      B          |    Capacity      |
| :----:   | :---------:   | :----------:    | :----------:     |
|  Labour  |      6        |      8          |     960          |
|  Machine |      9        |      7          |     870          |
|  Profit  |      10       |      11         |                  |

## Maximization of profit (Simple Linear Programming)

$$
\begin{aligned}
\max z = & 10x_1 + 11x_2 \\
\text{Subject to:} & \\
& 6x_1 + 8x_2 \le 960 \\
& 9x_1 + 7x_2 \le 870 \\
& x_1, x_2 \ge 0 \\
\end{aligned}
$$

In [2]:
using Pkg; Pkg.activate("."); using JuMP, HiGHS

  Activating project at `~/code/julia/notebooks`


In [3]:
m = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [4]:
@variable(m, x1 >= 0)

x1

In [5]:
@variable(m, x2 >= 0)

x2

In [6]:
@objective(m, Max, 10*x1 + 11*x2)

10 x1 + 11 x2

In [7]:
@constraint(m, 6*x1 + 8*x2 <= 960)

6 x1 + 8 x2 ≤ 960

In [8]:
@constraint(m, 9*x1 + 7*x2 <= 870)

9 x1 + 7 x2 ≤ 870

In [9]:
optimize!(m)

Running HiGHS 1.10.0 (git hash: fd8665394e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 2 rows; 2 cols; 4 nonzeros
Coefficient ranges:
  Matrix [6e+00, 9e+00]
  Cost   [1e+01, 1e+01]
  Bound  [0e+00, 0e+00]
  RHS    [9e+02, 1e+03]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 2(-0); elements 4(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.6249964556e+00 Ph1: 2(3.75); Du: 2(2.625) 0s
          2     1.3340000000e+03 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  1.3340000000e+03
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00


In [10]:
value.([x1, x2])

2-element Vector{Float64}:
   7.999999999999983
 114.00000000000001

In [11]:
objective_value(m)

1334.0

## Weighted Goal Programming

- Goal 1: Obtain at least 1320 of profit
- Goal 2: Produce at least 15 units from A
- The first goal is twice as asimportant as the second one

$$
\begin{aligned}
\min z = & 2d_1^- + d_2^- \\
\text{Subject to:} & \\
& 10x_1 + 11x_2 + d_1^- - d_1^+ = 1320 \\
& x_1 + d_2^- - d_2^+ = 15 \\
& 6x_1 + 8x_2 \le 960 \\
& 9x_1 + 7x_2 \le 870 \\
& x_1, x_2, d_1^-, d_1^+, d_2^-, d_2^+ \ge 0 \\
\end{aligned}
$$

In [20]:
begin 
    m = Model(HiGHS.Optimizer)
    @variable(m, x1 >= 0)
    @variable(m, x2 >= 0)
    @variable(m, d1minus >= 0)
    @variable(m, d1plus >= 0)
    @variable(m, d2minus >= 0)
    @variable(m, d2plus >= 0)
    @objective(m, Min, 2*d1minus + d2minus)
    @constraint(m, 10*x1 + 11*x2 + d1minus - d1plus == 1320)
    @constraint(m, x1 + d2minus - d2plus == 15)
    @constraint(m, 6*x1 + 8*x2 <= 960)
    @constraint(m, 9*x1 + 7*x2 <= 870)
    optimize!(m);
    println(value.([x1, x2, d1minus, d1plus, d2minus, d2plus]))
end 

Running HiGHS 1.10.0 (git hash: fd8665394e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 4 rows; 6 cols; 11 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [1e+00, 2e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+01, 1e+03]
Presolving model
4 rows, 6 cols, 11 nonzeros  0s
Dependent equations search running on 2 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
4 rows, 6 cols, 11 nonzeros  0s
Presolve : Reductions: rows 4(-0); columns 6(-0); elements 11(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(390) 0s
          3     3.6206896552e+00 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 3
Objective value     :  3.6206896552e+00
Relative P-D gap    :  1.2020014613e-14
HiGHS run time      :          0.00
[11.379310344827605, 1

- $d_1^- = 0$ and $d_1^+ = 0$ so Goal 1 is satisfied

- $d_2^- = 3.6206$ and $d_2^+ = 0$ so Goal 2 is **not** satisfied

## Pre-emptive Goal Programming

- Goal 1: Obtain at least 1320 of profit
- Goal 2: Produce at least 15 units from A
- Goal 2 is more important than the Goal 1

$$
\begin{aligned}
\min z = & d_2^- \\
\text{Subject to:} & \\
& 10x_1 + 11x_2 + d_1^- - d_1^+ = 1320 \\
& x_1 + d_2^- - d_2^+ = 15 \\
& 6x_1 + 8x_2 \le 960 \\
& 9x_1 + 7x_2 \le 870 \\
& x_1, x_2, d_1^-, d_1^+, d_2^-, d_2^+ \ge 0 \\
\end{aligned}
$$

In [21]:
begin 
    m = Model(HiGHS.Optimizer)
    @variable(m, x1 >= 0)
    @variable(m, x2 >= 0)
    @variable(m, d1minus >= 0)
    @variable(m, d1plus >= 0)
    @variable(m, d2minus >= 0)
    @variable(m, d2plus >= 0)
    @objective(m, Min, d2minus)
    @constraint(m, 10*x1 + 11*x2 + d1minus - d1plus == 1320)
    @constraint(m, x1 + d2minus - d2plus == 15)
    @constraint(m, 6*x1 + 8*x2 <= 960)
    @constraint(m, 9*x1 + 7*x2 <= 870)
    optimize!(m);
    println(value.([x1, x2, d1minus, d1plus, d2minus, d2plus]))
end 

Running HiGHS 1.10.0 (git hash: fd8665394e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 4 rows; 6 cols; 11 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+01, 1e+03]
Presolving model
4 rows, 6 cols, 11 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-4); columns 0(-6); elements 0(-11) - Reduced to empty
Solving the original LP from the solution after postsolve
Model status        : Optimal
Objective value     :  0.0000000000e+00
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00
[96.66666666666667, 0.0, 353.33333333333326, 0.0, 0.0, 81.66666666666666]


- $d_2^-$ is zero. Now substitude the value in the new model

$$
\begin{aligned}
\min z = & d_1^- \\
\text{Subject to:} & \\
& 10x_1 + 11x_2 + d_1^- - d_1^+ = 1320 \\
& x_1 - d_2^+ = 15 \\
& 6x_1 + 8x_2 \le 960 \\
& 9x_1 + 7x_2 \le 870 \\
& x_1, x_2, d_1^-, d_1^+, d_2^-, d_2^+ \ge 0 \\
\end{aligned}
$$

In [22]:
begin 
    m = Model(HiGHS.Optimizer)
    @variable(m, x1 >= 0)
    @variable(m, x2 >= 0)
    @variable(m, d1minus >= 0)
    @variable(m, d1plus >= 0)
    @variable(m, d2minus >= 0)
    @variable(m, d2plus >= 0)
    @objective(m, Min, d1minus)
    @constraint(m, 10*x1 + 11*x2 + d1minus - d1plus == 1320)
    @constraint(m, x1 - d2plus == 15)
    @constraint(m, 6*x1 + 8*x2 <= 960)
    @constraint(m, 9*x1 + 7*x2 <= 870)
    optimize!(m);
    println(value.([x1, x2, d1minus, d1plus, d2minus, d2plus]))
end 

Running HiGHS 1.10.0 (git hash: fd8665394e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 4 rows; 6 cols; 10 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+01]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+01, 1e+03]
Presolving model
3 rows, 4 cols, 8 nonzeros  0s
Dependent equations search running on 1 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
3 rows, 4 cols, 8 nonzeros  0s
Presolve : Reductions: rows 3(-1); columns 4(-2); elements 8(-2)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     7.9210135324e-05 Pr: 1(1170) 0s
          2     1.5000000000e+01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  1.5000000000e+01
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00
[15

- $d_1^- = 15$ and $d_1^+ = 0$ so Goal 1 is **not** satisfied
- $d_2^- = 0$ and $d_2^+ = 0$ so Goal 2 is satisfied